<center><h1>Using FBProphet Model</h1></center>

In [ ]:
import pandas as pd
from fbprophet import Prophet
import numpy as np

In [ ]:
#Preprocessing training data
train = pd.read_csv('../train.csv')
train['DateTime'] = train.UPDATE_TIME.map(str) + " " + train.HOUR_ID.map(str) + ":00"
train.DateTime = pd.to_datetime(train.DateTime)
train.DateTime = train.DateTime.dt.strftime("Y%-m%-d% %H:%M")

#Preprocessing testing data
test = pd.read_csv('../test_id.csv')
test['DateTime'] = test.UPDATE_TIME.map(str) + " " + test.HOUR_ID.map(str) + ":00"
test.DateTime = pd.to_datetime(test.DateTime)
test.DateTime = test.DateTime.dt.strftime("Y%-m%-d% %H:%M")
test['And'] = test['DateTime'].map(str) + test.SERVER_NAME.map(str)

server_name = test.SERVER_NAME.unique()
output_feature = ['BANDWIDTH_TOTAL','MAX_USER']

In [ ]:
for y1 in output_feature:
    print(y1)
    resultFinal = pd.DataFrame(None, columns = [y1,y1 +'_lower',y1 +'_upper','And'])

    for item in server_name:
        data_one = train[train.SERVER_NAME == item][['DateTime',y1]]
        print(item, data_one.shape)
        model = Prophet(yearly_seasonality=True)
        model.fit(data_one.reset_index().rename(columns={'DateTime':'ds', y1:'y'}))
        future = model.make_future_dataframe(periods=744, freq='h')
        
        forecast = model.predict(future)
        
        result = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
        result['And'] = result.ds.map(str) + item
        
        result = result.rename(columns={'yhat':y1,'yhat_lower':y1+'_lower','yhat_upper':y1+'_upper'})
        result = result[result.ds >= '2019-03-10']
        result = result.drop('ds', axis=1)
        resultFinal = pd.concat([resultFinal, result])
        print('resultFinal', resultFinal.shape)
        
    resultFinal.to_csv('FBProphet_' + y1 + ".csv")
    del resultFinal

In [ ]:
bandwidth = pd.read_csv('FBProphet_BANDWIDTH_TOTAL.csv')
user = pd.read_csv('FBProphet_MAX_USER.csv')
lookup_bandwidth = bandwidth[['And', 'BANDWIDTH_TOTAL']]
lookup_user = user[['And', 'MAX_USER']]
test = test.merge(lookup_bandwidth, on='And', how='left')
test = test.merge(lookup_user, on='And', how='left')

#Fill NaN
test[test == ""] = np.NaN
test = test.fillna(method='ffill')

#Trick of MAPE
test.BANDWIDTH_TOTAL[test.BANDWIDTH_TOTAL < 260] = 0
test.MAX_USER[test.MAX_USER < 1] = 0

#Make label value
test['label'] = round(test.BANDWIDTH_TOTAL,2).map(str) + " " + round(test.MAX_USER,2).map(str)

#Export data
test[['id','label']].to_csv('FBProphet_submit.csv', index=False)